In [ ]:
import pandas as pd
import numpy as np

from lib.feature_eng import find_class_columns, create_pivot_table, create_iv_pivot_table_with_binning, calculate_woe_iv
from lib.charts_config import plot_woe_iv

## Extraição de dados usando Kaggle CLI


In [ ]:
%%bash
mkdir -p data && \
cd data && \ 
kaggle competitions download -c home-credit-credit-risk-model-stability && \
unzip home-credit-credit-risk-model-stability.zip && \
rm -rf parquet_files \

## Seleção de arquivos relacionados com as classes escolhidas

In [35]:
train_depth_0 = ['train_static_0_0', 'train_static_0_1','train_static_cb_0']
train_depth_1 = [
  'train_applprev_1_0', 'train_applprev_1_1','train_other_1', 'train_tax_registry_a_1', 
  'train_tax_registry_b_1', 'train_tax_registry_c_1', 'train_credit_bureau_a_1_0', 
  'train_credit_bureau_a_1_1', 'train_credit_bureau_a_1_2', 'train_credit_bureau_a_1_3', 
  'train_credit_bureau_b_1', 'train_deposit_1', 'train_person_1', 'train_debitcard_1',
]
train_depth_2 = [
  'train_applprev_2', 'train_person_2', 'train_credit_bureau_a_2_0', 'train_credit_bureau_a_2_1',
  'train_credit_bureau_a_2_2', 'train_credit_bureau_a_2_3', 'train_credit_bureau_a_2_4',
  'train_credit_bureau_a_2_5', 'train_credit_bureau_a_2_6', 'train_credit_bureau_a_2_7',
  'train_credit_bureau_a_2_8', 'train_credit_bureau_a_2_9', 'train_credit_bureau_a_2_10',
  'train_credit_bureau_b_2',
]

classes_list = [
  'amount_416A', 'amtdepositbalance_4809441A','amtdepositincoming_4809444A', 'amtdepositoutgoing_4809442A',
  'amtinstpaidbefduel24m_4187115A', 'days360_512L','maxdpdlast12m_727P',
  'birthdate_87D', 'cntpmts24_3658933L', 'credacc_actualbalance_314A','credacc_maxhisbal_375A', 'credacc_minhisbal_90A',
  'currdebt_22A', 'currdebtcredtyperange_828A', 'contractsum_5085717L', 'downpmt_116A', 'education_88M',
  'for3years_128L', 'gender_992L', 'downpmt_134A'
]

depth_0_occurrences = find_class_columns(feature_cols=classes_list, file_list=train_depth_0)
depth_1_occurrences = find_class_columns(feature_cols=classes_list, file_list=train_depth_1)
depth_2_occurrences = find_class_columns(feature_cols=classes_list, file_list=train_depth_2)

# Dicionarios com arquivos e suas respectivas colunas (classes)
print(f'depth_0 = {depth_0_occurrences}') 
print(f'depth_1 = {depth_1_occurrences}')
print(f'depth_2 = {depth_2_occurrences}')

depth_0 = {'train_static_0_0': {'downpmt_116A', 'amtinstpaidbefduel24m_4187115A', 'cntpmts24_3658933L', 'currdebtcredtyperange_828A', 'currdebt_22A', 'maxdpdlast12m_727P'}, 'train_static_0_1': {'downpmt_116A', 'amtinstpaidbefduel24m_4187115A', 'cntpmts24_3658933L', 'currdebtcredtyperange_828A', 'currdebt_22A', 'maxdpdlast12m_727P'}, 'train_static_cb_0': {'for3years_128L', 'days360_512L', 'education_88M'}}
depth_1 = {'train_applprev_1_0': {'credacc_minhisbal_90A', 'downpmt_134A', 'credacc_maxhisbal_375A', 'credacc_actualbalance_314A'}, 'train_applprev_1_1': {'credacc_minhisbal_90A', 'downpmt_134A', 'credacc_maxhisbal_375A', 'credacc_actualbalance_314A'}, 'train_other_1': {'amtdepositincoming_4809444A', 'amtdepositbalance_4809441A', 'amtdepositoutgoing_4809442A'}, 'train_credit_bureau_a_1_0': {'contractsum_5085717L'}, 'train_credit_bureau_a_1_1': {'contractsum_5085717L'}, 'train_credit_bureau_a_1_2': {'contractsum_5085717L'}, 'train_credit_bureau_a_1_3': {'contractsum_5085717L'}, 'train_

## Pré-processamento dos dados selecionados

In [ ]:
df_base_info = pd.read_csv('data/csv_files/train/train_base.csv', usecols=['case_id', 'target'])
df_static_0_0 ='data/csv_files/train/train_static_0_0.csv'
df_static_0_1 ='data/csv_files/train/train_static_0_1.csv'
df_static_cb_0 = 'data/csv_files/train/train_static_cb_0.csv'

df = pd.read_csv(df_static_cb_0)
df

### Variável : `cntpmts24_3658933L`

Descrição : Quantidade de meses com qualquer recebimento (renda)

Classe IV : 0.0089

In [ ]:
# Create pivot table
cls = 'cntpmts24_3658933L'
df_base_cnt = create_pivot_table(df_base_info, df_static_0_0, cls)

# Using classes in range <12, 12-20 and >20
df_base_cnt['range'] = pd.cut(df_base_cnt.index, 
                     bins=[-float('inf'), 12, 20, float('inf')],
                     labels=['<12', '12-20', '>20'])

# Group by the ranges and sum the counts
df_class = df_base_cnt.groupby('range').agg({0: 'sum', 1: 'sum'})

# Rename columns for clarity
df_class.columns = ['non_default', 'default']

# Calculate total observations
df_class['total_obs'] = df_class['non_default'] + df_class['default']

# Calculate percentages
total_defaults = df_class['default'].sum()
total_non_defaults = df_class['non_default'].sum()

df_class['pct_events'] = df_class['default'] / total_defaults
df_class['pct_non_events'] = df_class['non_default'] / total_non_defaults

# Calculate WOE and IV
df_class['woe'] = np.log(df_class['pct_non_events'] / df_class['pct_events'])
df_class['iv'] = (df_class['pct_non_events'] - df_class['pct_events']) * df_class['woe']

print(df_class["iv"].sum())
df_class

### Variável : `amtinstpaidbefduel24m_4187115A`

Descrição : Número de parcelas pagas antes do vencimento nos últimos 24 meses

Classe IV : 0,031

In [ ]:
# Create pivot table
cls = 'amtinstpaidbefduel24m_4187115A'
df_class_amt = create_iv_pivot_table_with_binning(df_base_info, df_static_0_0, cls, num_bins=40)

print(df_class_amt["iv"].sum())

df_class_amt

In [ ]:
# Data Vizualization
plot_woe_iv(df_class_amt, cls, rotation_X=90)

### Variável : `currdebt_22A`

Descrição : Atual total de debido do cliente 

Classe IV : 0,002

In [ ]:
cls = 'currdebt_22A'
# df_class_curr = create_pivot_table(df_base_info, df_static_0_0, cls, )
df_class_curr = create_iv_pivot_table_with_binning(df_base_info, df_static_0_0, cls, num_bins=20)

print(df_class_curr["iv"].sum())

df_class_curr

In [ ]:
# Data Vizualization
plot_woe_iv(df_class_curr, cls, rotation_X=90)

### Variável : `maxdpdlast12m_727P`

Descrição : Valor total de outros contratos

Classe IV : 0,12

In [ ]:
cls = 'maxdpdlast12m_727P'
# df_class_max_dpd_year = create_pivot_table(df_base_info, df_static_0_0, cls, )
df_class_max_dpd_year = create_iv_pivot_table_with_binning(df_base_info, df_static_0_0, cls, num_bins=2000)

print(df_class_max_dpd_year["iv"].sum())

df_class_max_dpd_year

In [ ]:
# Data Vizualization
plot_woe_iv(df_class_max_dpd_year, cls, rotation_X=90)

### Variável : `education_88M`

Descrição : Nivel educacional

Classe IV : 0,003

In [ ]:
cls = 'education_88M'
df_class_education = create_pivot_table(df_base_info, df_static_cb_0, cls)
df_class_edication['ta']
df_education_iv = calculate_woe_iv(df_class_education)
df

print(df_education_iv["iv"].sum())

df_education_iv

### Variável : `contractssum_5085716L`

Descrição : Soma total de valores de contratos recuperados de bureau de crédito externo

Classe IV : 0,021

In [ ]:
cls = 'contractssum_5085716L'
df_class_retrieved_contracts = create_iv_pivot_table_with_binning(df_base_info, df_static_cb_0, cls, num_bins=2000)

print(df_class_retrieved_contracts["iv"].sum())

df_class_retrieved_contracts

In [ ]:
# Data Vizualization
plot_woe_iv(df_class_retrieved_contracts, cls, rotation_X=90)

### Variável : `downpmt_116A`

Descrição : Valor da entrada

Classe IV : 0,0067

In [ ]:
cls = 'downpmt_116A'
df_class_downpay = create_iv_pivot_table_with_binning(df_base_info, df_static_0_0, cls, num_bins=2000)

print(df_class_downpay["iv"].sum())

df_class_downpay

In [ ]:
# Data Vizualization
plot_woe_iv(df_class_downpay, cls, rotation_X=90)